In [478]:
#Import dependencies.
import pandas as pd
import numpy as np
import os
import csv
import warnings
warnings.filterwarnings('ignore')
from io import StringIO
import zipfile
from pathlib import Path

# Population Data Cleaning.

In [479]:
#Read dataframe.
conflict_df=pd.read_csv("Resources/cleaned conflict.csv",header=0)
demographics_df=pd.read_csv("Resources/demographics.csv",header=0)
disasters_df=pd.read_csv("Resources/cleaned disasters.csv",header=0)
decision_df=pd.read_csv("Resources/cleaned_asylum.csv",header=0)

In [480]:
country_list = demographics_df['Country of origin'].unique().tolist()

In [481]:
conflict_df['helper'] = conflict_df['Year'].astype(str) + conflict_df['Country of origin']
demographics_df['helper'] = demographics_df['Year'].astype(str) + demographics_df['Country of origin']
disasters_df['helper'] = disasters_df['Year'].astype(str) + disasters_df['Country of origin']
decision_df['helper'] = decision_df['Year'].astype(str) + decision_df['Country of origin']
demographics_df

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,...,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Male total,Total,helper
0,2001,Iran (Islamic Rep. of),IRN,Afghanistan,AFG,0,0,0,0,0,...,0,0,0,0,5,0,0,5,5,2001Iran (Islamic Rep. of)
1,2001,Iraq,IRQ,Afghanistan,AFG,0,5,0,0,0,...,5,0,0,0,0,0,0,0,5,2001Iraq
2,2001,Iraq,IRQ,Albania,ALB,0,0,0,0,0,...,0,0,0,0,7,0,0,7,7,2001Iraq
3,2001,North Macedonia,MKD,Albania,ALB,0,0,0,6,0,...,6,0,0,0,0,0,0,0,6,2001North Macedonia
4,2001,Serbia and Kosovo: S/RES/1244 (1999),SRB,Albania,ALB,14,20,18,89,8,...,149,14,18,16,45,18,0,111,278,2001Serbia and Kosovo: S/RES/1244 (1999)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74568,2021,Rwanda,RWA,Zimbabwe,ZWE,29,48,49,179,14,...,319,25,54,51,188,23,0,341,660,2021Rwanda
74569,2021,Somalia,SOM,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,13,0,0,13,13,2021Somalia
74570,2021,Sudan,SDN,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,8,0,0,8,8,2021Sudan
74571,2021,South Sudan,SSD,Zimbabwe,ZWE,0,0,0,0,0,...,0,0,0,0,8,0,0,8,8,2021South Sudan


In [482]:
# Group the data by Country of origin and Year and calculate the sum of the Total
summed_df = demographics_df.groupby(['Country of origin', 'Country of asylum', 'Year', 'helper', 'Female 0 - 4', 'Female 5 - 11', 'Female 12 - 17', 'Female 18 - 59', 'Female 60', 'Female other', 'Female total', 'Male 0 - 4', 'Male 5 - 11', 'Male 12 - 17', 'Male 18 - 59', 'Male 60', 'Male other', 'Male total'], as_index=False)['Total'].sum()

# Group by Country of origin to get the full count of every year and drop if the Total count is below 50,000
filtered_df = summed_df.groupby('Country of origin').filter(lambda x: x['Total'].sum() >= 50000)

# Reset the index to ungroup the data
demo_result_df = pd.DataFrame(filtered_df.reset_index(drop=True))

In [483]:
grouped_conflict_df = conflict_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_demo_result_df = demo_result_df.groupby(['Year', 'Country of origin','Country of asylum', 'helper']).sum()
grouped_disasters_df = disasters_df.groupby(['Year', 'Country of origin', 'helper']).sum()
grouped_decision_df = decision_df.groupby(['Year', 'Country of origin', 'Country of asylum','helper']).sum()
grouped_conflict_df = pd.DataFrame(grouped_conflict_df.reset_index())
grouped_demo_result_df = pd.DataFrame(grouped_demo_result_df.reset_index())
grouped_disasters_df = pd.DataFrame(grouped_disasters_df.reset_index())
grouped_decision_df = pd.DataFrame(grouped_decision_df.reset_index())
grouped_decision_df

,Year,Country of origin,Country of asylum,helper,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2001,Afghanistan,Armenia,2001Afghanistan,0,0,5,0,5
1,2001,Afghanistan,Australia,2001Afghanistan,1914,0,522,10,2446
2,2001,Afghanistan,Austria,2001Afghanistan,432,0,335,0,767
3,2001,Afghanistan,Azerbaijan,2001Afghanistan,226,0,45,0,271
4,2001,Afghanistan,Belarus,2001Afghanistan,85,0,25,51,161
...,...,...,...,...,...,...,...,...,...
63706,2021,Zimbabwe,Sweden,2021Zimbabwe,0,0,16,0,16
63707,2021,Zimbabwe,Switzerland,2021Zimbabwe,0,0,5,0,5
63708,2021,Zimbabwe,Ukraine,2021Zimbabwe,0,0,0,5,5
63709,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,2021Zimbabwe,44,5,53,10,112


In [484]:
new_country_list = grouped_demo_result_df['Country of origin'].unique().tolist()
print(len(new_country_list))

88


In [485]:
dropped_countries = list(set(country_list) - set(new_country_list))
dropped_countries.sort()
print(len(dropped_countries))

113


In [486]:

filtered_decision_df = grouped_decision_df[['Year','Country of origin','Country of asylum','helper','Recognized decisions','Complementary protection','Rejected decisions','Otherwise closed', 'Total decisions']]
filtered_demographics_df = grouped_demo_result_df[['Year','Country of origin', 'Country of asylum','helper','Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other','Total']]
filtered_conflict_df = grouped_conflict_df[['Year','Country of origin','helper', 'Deaths civilians']]
filtered_disasters_df = grouped_disasters_df[['Year','Country of origin','helper','Total deaths','Total affected']]
filtered_demographics_df

,Year,Country of origin,Country of asylum,helper,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male other,Total
0,2001,Afghanistan,Australia,2001Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,6262
1,2001,Afghanistan,Austria,2001Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,1049
2,2001,Afghanistan,Azerbaijan,2001Afghanistan,8,12,10,38,0,0,9,26,22,118,0,0,243
3,2001,Afghanistan,Belarus,2001Afghanistan,9,40,36,81,6,0,9,34,35,199,5,0,454
4,2001,Afghanistan,Belgium,2001Afghanistan,0,0,0,0,0,144,0,0,0,0,0,213,357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61987,2021,Zimbabwe,Sweden,2021Zimbabwe,0,0,0,0,0,0,0,0,0,0,0,0,16
61988,2021,Zimbabwe,Switzerland,2021Zimbabwe,0,9,0,0,0,0,0,0,0,0,0,0,9
61989,2021,Zimbabwe,Thailand,2021Zimbabwe,0,0,0,0,0,0,9,0,0,0,0,0,9
61990,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,2021Zimbabwe,0,0,0,0,0,0,0,0,0,0,0,0,1174


In [487]:
filtered_decision_df

,Year,Country of origin,Country of asylum,helper,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2001,Afghanistan,Armenia,2001Afghanistan,0,0,5,0,5
1,2001,Afghanistan,Australia,2001Afghanistan,1914,0,522,10,2446
2,2001,Afghanistan,Austria,2001Afghanistan,432,0,335,0,767
3,2001,Afghanistan,Azerbaijan,2001Afghanistan,226,0,45,0,271
4,2001,Afghanistan,Belarus,2001Afghanistan,85,0,25,51,161
...,...,...,...,...,...,...,...,...,...
63706,2021,Zimbabwe,Sweden,2021Zimbabwe,0,0,16,0,16
63707,2021,Zimbabwe,Switzerland,2021Zimbabwe,0,0,5,0,5
63708,2021,Zimbabwe,Ukraine,2021Zimbabwe,0,0,0,5,5
63709,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,2021Zimbabwe,44,5,53,10,112


In [488]:
filtered_conflict_df

,Year,Country of origin,helper,Deaths civilians
0,2001,Afghanistan,2001Afghanistan,864
1,2001,Algeria,2001Algeria,32
2,2001,Angola,2001Angola,1038
3,2001,Azerbaijan,2001Azerbaijan,0
4,2001,Bangladesh,2001Bangladesh,6
...,...,...,...,...
1092,2021,Turkey,2021Turkey,0
1093,2021,Uganda,2021Uganda,7
1094,2021,Ukraine,2021Ukraine,18
1095,2021,Venezuela,2021Venezuela,19


In [489]:
filtered_conflict_df = filtered_conflict_df.rename(columns={'Deaths civilians': 'conflict_deaths'})
filtered_conflict_df

,Year,Country of origin,helper,conflict_deaths
0,2001,Afghanistan,2001Afghanistan,864
1,2001,Algeria,2001Algeria,32
2,2001,Angola,2001Angola,1038
3,2001,Azerbaijan,2001Azerbaijan,0
4,2001,Bangladesh,2001Bangladesh,6
...,...,...,...,...
1092,2021,Turkey,2021Turkey,0
1093,2021,Uganda,2021Uganda,7
1094,2021,Ukraine,2021Ukraine,18
1095,2021,Venezuela,2021Venezuela,19


In [490]:
grouped_disasters_df.head()

,Year,Country of origin,helper,Total deaths,Total affected
0,2001,Afghanistan,2001Afghanistan,331.0,200270.0
1,2001,Algeria,2001Algeria,921.0,45423.0
2,2001,Angola,2001Angola,48.0,39928.0
3,2001,Argentina,2001Argentina,15.0,253000.0
4,2001,Australia,2001Australia,3.0,3001.0


In [491]:
filtered_disasters_df = filtered_disasters_df.rename(columns={'Total deaths': 'disaster_deaths', 'Total affected':'disaster_affected'})
filtered_disasters_df 

,Year,Country of origin,helper,disaster_deaths,disaster_affected
0,2001,Afghanistan,2001Afghanistan,331.0,200270.0
1,2001,Algeria,2001Algeria,921.0,45423.0
2,2001,Angola,2001Angola,48.0,39928.0
3,2001,Argentina,2001Argentina,15.0,253000.0
4,2001,Australia,2001Australia,3.0,3001.0
...,...,...,...,...,...
1477,2021,United States of America,2021United States of America,494.0,17059.0
1478,2021,Uzbekistan,2021Uzbekistan,8.0,6.0
1479,2021,Viet Nam,2021Viet Nam,9.0,83430.0
1480,2021,Yemen,2021Yemen,13.0,22380.0


In [492]:
#filtered_demographics_df

#adding all the male and female by age totals. 
filtered_demographics_df['MaleFemaleTotal'] = filtered_demographics_df[['Female 0 - 4','Female 5 - 11','Female 12 - 17','Female 18 - 59','Female 60','Female other','Male 0 - 4','Male 5 - 11','Male 12 - 17','Male 18 - 59','Male 60','Male other']].sum(axis=1)
#creating an Unknown demographic calculated field. 
filtered_demographics_df['unknown_demographic'] = filtered_demographics_df['Total'] - filtered_demographics_df['MaleFemaleTotal']
col_to_move = filtered_demographics_df.pop('unknown_demographic')  
filtered_demographics_df.insert(16, 'unknown_demographic', col_to_move)
filtered_demographics_df = filtered_demographics_df.drop('helper', axis=1)
filtered_demographics_df = filtered_demographics_df.drop('MaleFemaleTotal', axis=1)
filtered_demographics_df=filtered_demographics_df.replace(to_replace="TÃ¼rkiye",
           value="Turkey")
demographic_ml_df = filtered_demographics_df

demographic_ml_df = pd.merge(demographic_ml_df, filtered_decision_df , on=['Year', 'Country of origin', 'Country of asylum'], how='left')

In [495]:
demographic_ml_df

,Year,Country of origin,Country of asylum,Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female other,Male 0 - 4,...,Male 18 - 59,Male 60,Male other,unknown_demographic,Total,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2001,Afghanistan,Australia,0,0,0,0,0,0,0,...,0,0,0,6262,6262,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,0,0,0,0,0,0,0,...,0,0,0,1049,1049,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,8,12,10,38,0,0,9,...,118,0,0,0,243,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,9,40,36,81,6,0,9,...,199,5,0,0,454,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,0,0,0,0,0,144,0,...,0,0,213,0,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61987,2021,Zimbabwe,Sweden,0,0,0,0,0,0,0,...,0,0,0,16,16,0.0,0.0,16.0,0.0,16.0
61988,2021,Zimbabwe,Switzerland,0,9,0,0,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61989,2021,Zimbabwe,Thailand,0,0,0,0,0,0,9,...,0,0,0,0,9,NaN,NaN,NaN,NaN,NaN
61990,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,0,...,0,0,0,1174,1174,44.0,5.0,53.0,10.0,112.0


In [496]:
#data merged by year totals. 

merged = pd.merge(filtered_conflict_df, filtered_disasters_df , on=['Year', 'Country of origin','helper'], how='left')
merged = merged.drop('helper', axis=1)
col_to_move = merged.pop('conflict_deaths')  
merged.insert(4, 'conflict_deaths', col_to_move)
for col in merged.columns:
    if merged[col].dtype == float:
        merged[col] = merged[col].fillna(0).astype(int)
event_ml_df = merged
event_ml_df

,Year,Country of origin,disaster_deaths,disaster_affected,conflict_deaths
0,2001,Afghanistan,331,200270,864
1,2001,Algeria,921,45423,32
2,2001,Angola,48,39928,1038
3,2001,Azerbaijan,0,0,0
4,2001,Bangladesh,232,530150,6
...,...,...,...,...,...
1092,2021,Turkey,9,561088,0
1093,2021,Uganda,0,0,7
1094,2021,Ukraine,1,1700,18
1095,2021,Venezuela,0,0,19


In [497]:
demographic_ml_df = demographic_ml_df.rename(columns={'Year' : 'year',
                                                    'Country of origin' : 'country_origin',	
                                                    'Country of asylum_x' : 'country_asylum',	
                                                    'Female 0 - 4' : 'female_0to4',	
                                                    'Female 5 - 11' : 'female_5to11',	
                                                    'Female 12 - 17' : 'female_12to17',	
                                                    'Female 18 - 59' : 'female_18to59',	
                                                    'Female 60' :	'female_60',
                                                    'Female other' : 'female_other',	
                                                    'Male 0 - 4' : 'male_0to4',		
                                                    'Male 5 - 11' : 'male_5to11',
                                                    'Male 12 - 17' : 'male_12to17',	
                                                    'Male 18 - 59' : 'male_18to59',	
                                                    'Male 60' : 'male_60',	
                                                    'Male other' : 'male_other',
                                                    'Total' : 'total',
                                                    'Recognized decisions':'recognized_decisions',
                                                    'Complementary protection':'complementary_protection',
                                                    'Rejected decisions':'rejected',
                                                    'Otherwise closed':'otherwise_closed',
                                                    'Total decisions':'total_decisions'
    
})
demographic_ml_df

,year,country_origin,Country of asylum,female_0to4,female_5to11,female_12to17,female_18to59,female_60,female_other,male_0to4,...,male_18to59,male_60,male_other,unknown_demographic,total,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,2001,Afghanistan,Australia,0,0,0,0,0,0,0,...,0,0,0,6262,6262,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,0,0,0,0,0,0,0,...,0,0,0,1049,1049,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,8,12,10,38,0,0,9,...,118,0,0,0,243,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,9,40,36,81,6,0,9,...,199,5,0,0,454,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,0,0,0,0,0,144,0,...,0,0,213,0,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61987,2021,Zimbabwe,Sweden,0,0,0,0,0,0,0,...,0,0,0,16,16,0.0,0.0,16.0,0.0,16.0
61988,2021,Zimbabwe,Switzerland,0,9,0,0,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61989,2021,Zimbabwe,Thailand,0,0,0,0,0,0,9,...,0,0,0,0,9,NaN,NaN,NaN,NaN,NaN
61990,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,0,...,0,0,0,1174,1174,44.0,5.0,53.0,10.0,112.0


In [498]:
demographic_ml_df

,year,country_origin,Country of asylum,female_0to4,female_5to11,female_12to17,female_18to59,female_60,female_other,male_0to4,...,male_18to59,male_60,male_other,unknown_demographic,total,recognized_decisions,complementary_protection,rejected,otherwise_closed,total_decisions
0,2001,Afghanistan,Australia,0,0,0,0,0,0,0,...,0,0,0,6262,6262,1914.0,0.0,522.0,10.0,2446.0
1,2001,Afghanistan,Austria,0,0,0,0,0,0,0,...,0,0,0,1049,1049,432.0,0.0,335.0,0.0,767.0
2,2001,Afghanistan,Azerbaijan,8,12,10,38,0,0,9,...,118,0,0,0,243,226.0,0.0,45.0,0.0,271.0
3,2001,Afghanistan,Belarus,9,40,36,81,6,0,9,...,199,5,0,0,454,85.0,0.0,25.0,51.0,161.0
4,2001,Afghanistan,Belgium,0,0,0,0,0,144,0,...,0,0,213,0,357,100.0,0.0,42.0,0.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61987,2021,Zimbabwe,Sweden,0,0,0,0,0,0,0,...,0,0,0,16,16,0.0,0.0,16.0,0.0,16.0
61988,2021,Zimbabwe,Switzerland,0,9,0,0,0,0,0,...,0,0,0,0,9,0.0,0.0,5.0,0.0,5.0
61989,2021,Zimbabwe,Thailand,0,0,0,0,0,0,9,...,0,0,0,0,9,NaN,NaN,NaN,NaN,NaN
61990,2021,Zimbabwe,United Kingdom of Great Britain and Northern I...,0,0,0,0,0,0,0,...,0,0,0,1174,1174,44.0,5.0,53.0,10.0,112.0


In [499]:
event_ml_df = event_ml_df.rename(columns={'Year':'year',
                                          'Country of origin': 'country_origin',
                                        })
event_ml_df

,year,country_origin,disaster_deaths,disaster_affected,conflict_deaths
0,2001,Afghanistan,331,200270,864
1,2001,Algeria,921,45423,32
2,2001,Angola,48,39928,1038
3,2001,Azerbaijan,0,0,0
4,2001,Bangladesh,232,530150,6
...,...,...,...,...,...
1092,2021,Turkey,9,561088,0
1093,2021,Uganda,0,0,7
1094,2021,Ukraine,1,1700,18
1095,2021,Venezuela,0,0,19


In [501]:
demographic_ml_df.to_csv('Resources/demographic_ml_df.csv')
event_ml_df.to_csv('Resources/event_ml_df.csv')